In [ ]:
%pip install geometrix

Found existing installation: geometrix 0.1.12
Uninstalling geometrix-0.1.12:
  Successfully uninstalled geometrix-0.1.12
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys, site, importlib.util
print("exe:", sys.executable)
print("site:", site.getsitepackages())
print("user site:", site.getusersitepackages())
print("spec:", importlib.util.find_spec("geometrix"))
!{sys.executable} -m pip show geometrix

exe: /home/mcesel/Documents/proj/geometrix/.venv/bin/python
site: ['/home/mcesel/Documents/proj/geometrix/.venv/lib/python3.11/site-packages']
user site: /home/mcesel/.local/lib/python3.11/site-packages
spec: None
Name: geometrix
Version: 0.1.12
Summary: LaTeX-first differential geometry renderer for Jupyter/Colab.
Home-page: https://github.com/0kilo/geometrix.git
Author: 
Author-email: Khan Tran <khanster408@gmail.com>
License: MIT
Location: /home/mcesel/Documents/proj/geometrix/.venv/lib/python3.11/site-packages
Requires: einsteinpy, ipywidgets, litellm, numba, numexpr, numpy, pydantic, scikit-image, sympy, traitlets
Required-by: 


In [3]:
import os

import sympy as sp
# from google.colab import userdata # For use with Colab

from geometrix import latex, llm_solve, show
from geometrix.api import SceneBundle

from geometrix.symbolic import LLMConfig
from geometrix.sample.domains import Domain
from geometrix.sample.surface import sample_surface_grid
from geometrix.scene.build import build_surface_scene
from geometrix.symbolic.compile import compile_vector

ModuleNotFoundError: No module named 'geometrix'

In [ ]:
config = LLMConfig(
    provider="gemini",
    model="gemini-3-flash-preview",
    api_key=os.getenv("GEOMETRIX_GEMINI_API_KEY")
    #api_key=userdata.get("GEMINI_API_KEY"), #For use with Colab
)
# LLM-assisted solve with full derivation + validation.
live = llm_solve(
    "z_{xx}+z_{yy}+2z=0",
    config=config,
    response_type="full",
    wants_graph=True,
    graph_dim=3,
)

# Use the graph expression (expects z = f(x,y)).
graph_latex = live.data.graph
lhs, rhs = graph_latex.split("=", 1)
rhs = rhs.strip()

# Parse the explicit surface from LaTeX.
x, y = sp.symbols("x y")
z_expr = latex(rhs)

# Graph domain for a wave surface.
domains = [Domain("x", 0.0, 2 * sp.pi.evalf()), Domain("y", 0.0, 2 * sp.pi.evalf())]
counts = [120, 120]

# Sample and render the surface.
compiled = compile_vector([x, y, z_expr], [x, y])
surface = sample_surface_grid(compiled, domains, counts)
scene = build_surface_scene(surface.positions, surface.grid_shape)
show(SceneBundle(scene=scene, arrays={"positions": surface.positions}))


In [ ]:
# --- LLM example: implicit sphere (no rendering yet) ---
implicit = llm_solve(
    "x^2 + y^2 + z^2 - 1 = 0",
    config=config,
    response_type="minimal",
    wants_graph=True,
    graph_dim=3,
)
print("graph_type:", implicit.data.graph_type)
print("graph:", implicit.data.graph)


In [ ]:
# --- LLM example: parametric surface (rendered) ---
parametric = llm_solve(
    "	ext{Give a parametric torus }(x(u,v),y(u,v),z(u,v))",
    config=config,
    response_type="minimal",
    wants_graph=True,
    graph_dim=3,
)
print("graph_type:", parametric.data.graph_type)
print("graph:", parametric.data.graph)

# Expect graph as (x(u,v), y(u,v), z(u,v)) or similar.
graph = parametric.data.graph.strip()
if graph.startswith("(") and graph.endswith(")"):
    inner = graph[1:-1]
    parts = [p.strip() for p in inner.split(",")]
    if len(parts) == 3:
        u, v = sp.symbols("u v")
        x_expr = latex(parts[0])
        y_expr = latex(parts[1])
        z_expr = latex(parts[2])
        compiled = compile_vector([x_expr, y_expr, z_expr], [u, v])
        domains = [Domain("u", 0.0, 2 * sp.pi.evalf()), Domain("v", 0.0, 2 * sp.pi.evalf())]
        surface = sample_surface_grid(compiled, domains, [120, 120])
        scene = build_surface_scene(surface.positions, surface.grid_shape)
        show(SceneBundle(scene=scene, arrays={"positions": surface.positions}))
    else:
        print("Unexpected parametric format; skipping render.")
else:
    print("Parametric graph not in tuple form; skipping render.")


In [ ]:
# --- LLM example: tensor metric derivation (full) ---
metric = llm_solve(
    "	ext{Derive the metric tensor in spherical coordinates}",
    config=config,
    response_type="full",
    wants_graph=False,
    graph_dim=3,
)
print("metric:", metric.data.solution)
